In [1]:
import pyodbc
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import paramiko
import os
import time
from datetime import datetime, timedelta
from optimus_ids import Optimus

In [ ]:
import os
import time
import warnings
import pyodbc
import pandas as pd
import numpy as np
from tqdm import tqdm
import paramiko
from datetime import datetime, timedelta
from IPython.display import clear_output
warnings.filterwarnings('ignore')

# Constants
SERVER_NAME=""
DATABASE=""
UID=""
PWD=""
BATCH_SIZE=2000
FILEMAN_IP=""
FILEMAN_USER=""
FILEMAN_PASSWORD=""
VISTA_USERNAME=""
VISTA_PASSWORD=""
FILEMAN_SETTING_FILE_PATH="fileman_patient_list_conditions.xlsx"
optimus_prime=""
optimus_random=""


my_optimus = Optimus(
    prime=optimus_prime,
    random=optimus_random
)


columns = [
    "MRN", "DATE_ESTABLISHED", "DATE_OF_DEATH", "DOB", "SEX", "DATE_OF_LAST_UPDATE","NATIONALITY","PROVINCE","FILE_ACTIVE","FILE_TEMPORARY","MARITAL_STATUS"]

def establish_connection():
    """Establish a connection to the SQL Server."""
    try:
        conn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};'
                              f'SERVER={SERVER_NAME};'
                              f'DATABASE={DATABASE}; UID={UID}; PWD={PWD};')
        print("Connection successful!")
        return conn
    except pyodbc.Error as e:
        print("Connection error:", e)
        return None

def get_max_value_from_db(conn):
    """Retrieve the LEAST value from GREATEST DATE_OF_LAST_UPDATE or DATE_OF_DEATH column in the patients table."""
    sql_get = "SELECT LEAST(MAX(GREATEST(DATE_OF_LAST_UPDATE, DATE_OF_DEATH))) AS least_greatest_date FROM VISTA_PATIENTS;"
    cursor = conn.cursor()
    cursor.execute(sql_get)
    max_value = cursor.fetchone()[0]
    cursor.close()
#     return max_value if max_value is not None else 32508760
    return pd.to_datetime(max_value,format="%m/%d/%Y")-pd.Timedelta(days=1) if max_value is not None else pd.to_datetime('01/01/2000',format="%m/%d/%Y")



def update_excel_with_max_value(file_path, max_value):
    """Update the Excel file with the maximum value retrieved from the database."""
    df = pd.read_excel(file_path, header=None)
    max_value=max_value.strftime("%m/%d/%Y")
#     max_value = max_value.replace(" ", "@", 1)
    df.iloc[3, 0] = max_value
    df.iloc[6, 0] = max_value
    return df

def generate_fileman_string(df):
    """Generate a string for the FileMan search based on the updated Excel file."""
    return ''.join(str(row[0]) + '\n' if not pd.isna(row[0]) else '\x0d' for _, row in df.iterrows())

def setup_ssh_connection(host, username, password, port=22):
    """Set up an SSH connection."""
    paramiko.util.log_to_file("patient_list.log")
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, username=username, password=password, port=port)
    return ssh


def truncate_string(value, max_length):
    """Truncate string to the specified maximum length."""
    if isinstance(value, str) and len(value) > max_length:
        return value[:max_length]
    return value
string_columns_max_length = {
        "SEX": 10,
        "NATIONALITY":20,
        "PROVINCE":50
    

    }
                   

def extract_data():
    """Main function to extract data by connecting to the database and performing SSH operations."""
    conn = establish_connection()
    if not conn:
        return
    
    global start_date
    global end_date
    max_value = get_max_value_from_db(conn)
    conn.close()
    print(f"The maximum value in the column DATE_OF_LAST_UPDATE or DATE_OF_DEATH is: {max_value}")
    start_date=pd.to_datetime(max_value,format="%m/%d/%Y")
    print(f"The start Date  is: {start_date}")
#     end_date=pd.to_datetime(datetime.now()-timedelta(minutes=2))
#     print(f"The end Date  is: {end_date}")
    
    fileman_search = update_excel_with_max_value(FILEMAN_SETTING_FILE_PATH, max_value)
    fileman = generate_fileman_string(fileman_search)
    
    steps = [
        '\x0d',
        VISTA_USERNAME,
        '\x0d',
        VISTA_PASSWORD,
        '\x0d',
        '\x0d',
        'Search File Entries\n',
        fileman
    ]

    ssh = setup_ssh_connection(FILEMAN_IP, FILEMAN_USER, FILEMAN_PASSWORD)
    channel = ssh.invoke_shell()

    if os.path.exists('output_patient_list.txt'):
        os.remove('output_patient_list.txt')
    
    
    for step in steps[:-1]:
        channel.send(step)
        while not channel.recv_ready():
            time.sleep(3)
        out = channel.recv(9999)
        print(out.decode('cp1256'))
        if 'invalid signon attempts.' in out.decode('cp1256') or 'Device/IP address is locked in' in out.decode('cp1256') or 'Do you really want to halt? YES//' in out.decode('cp1256') or 'Not a valid ACCESS CODE' in out.decode('cp1256') :
            print(f"Sorry, invalid signon attempts, i will retry after 1 min")
            time.sleep(60)
            main_function() 
    channel.send(steps[-1])
    prev_output = ""
    while True:
        out = channel.recv(9999).decode('cp1256')
        with open('output_patient_list.txt', 'a') as f:
            f.write(out)
        
        combined_output = prev_output + out
        prev_output = out

        if "MATCHES FOUND" in combined_output or "MATCH FOUND" in combined_output:
            ssh.close()
            break

    print("Data extraction ended.")
#     ssh.close()

def close_connection(conn):
    """Close the database connection."""
    if conn:
        conn.close()
        print("Connection closed.")

def parse_file(filename, conn):
    """Parse the file and insert data into the database."""
    data_list = []
    num_lines = sum(1 for line in open(filename))
    pbar = tqdm(total=num_lines)
    

    
    with open(filename) as file:
        parsed_data = {}
        for line in file:
            if ">>>>>><<<<<<" in line and "THEN PRINT" not in line and "FIRST PRINT" not in line:
                delimiter_index = line.find(">>>>>><<<<<<")
                pbar.update(1)
                if line[delimiter_index + 12:].strip() != "":
                    key = line[:delimiter_index].strip()
                    value = line[delimiter_index + 12:].strip()
                    if key == 'MRN' and len(value) == 10:
                        continue
                    parsed_data[key] = value
            elif line.startswith("End Text >>>>>>>>>>"):
                pbar.update(1)
                data_list.append(parsed_data)
                parsed_data = {}

                
#                 insert_data_(conn, data_list, columns)
#                 data_list.clear()
            else:
                pbar.update(1)
    return data_list
#     if data_list:
#         insert_data_(conn, data_list, columns)

    


def insert_data_(conn, data_list, columns):
    """Insert data into the database in batches."""
    df= pd.DataFrame(data_list,columns=columns)
    #################
    df['MRN'] = pd.to_numeric(df['MRN'],errors='coerce')
    df['MRN'].fillna(0, inplace=True)
    df['MRN'] = df['MRN'].astype(int)
    
    df['DATE_ESTABLISHED'] = pd.to_datetime(df['DATE_ESTABLISHED'],format='%m/%d/%Y',errors='coerce')
    df['DATE_OF_DEATH'] = pd.to_datetime(df['DATE_OF_DEATH'],format='%m/%d/%Y',errors='coerce')
    df['DOB'] = pd.to_datetime(df['DOB'],format='%m/%d/%Y',errors='coerce')
    df['DATE_OF_LAST_UPDATE'] = pd.to_datetime(df['DATE_OF_LAST_UPDATE'],format='%m/%d/%Y',errors='coerce')
    
    df['DATE_ESTABLISHED'].fillna(pd.to_datetime(np.nan), inplace=True)
    df['DATE_OF_DEATH'].fillna(pd.to_datetime(np.nan), inplace=True)
    df['DOB'].fillna(pd.to_datetime(np.nan), inplace=True)
    df['DATE_OF_LAST_UPDATE'].fillna(pd.to_datetime(np.nan), inplace=True)

    ###############
    
    df = df.where(pd.notnull(df), None)
    df = df.replace({np.nan: None})
    for col, max_length in string_columns_max_length.items():
         df[col] = df[col].apply(lambda x: truncate_string(x, max_length))
            
    if not df.empty:
        insert_count=0
        update_count=0
        cursor = conn.cursor()

        for row in tqdm(df.itertuples(index=False), total=len(df), desc="Processing rows"):

            cursor.execute("SELECT COUNT(1) FROM VISTA_PATIENTS WHERE MRN = ?", my_optimus.encode(row.MRN))
            exists = cursor.fetchone()[0]
            if exists:
                update_query = "UPDATE VISTA_PATIENTS SET DATE_ESTABLISHED=?, DATE_OF_DEATH=?, DOB=?, SEX=?, DATE_OF_LAST_UPDATE=? ,NATIONALITY=? ,PROVINCE=?,FILE_ACTIVE=? ,FILE_TEMPORARY=? ,MARITAL_STATUS=? WHERE MRN=?;"
                cursor.execute(update_query, (row.DATE_ESTABLISHED, row.DATE_OF_DEATH, row.DOB, row.SEX, row.DATE_OF_LAST_UPDATE,row.NATIONALITY,row.PROVINCE,row.FILE_ACTIVE,row.FILE_TEMPORARY,row.MARITAL_STATUS,my_optimus.encode(row.MRN)))
         
                update_count+=1
                
            else:
                insert_query = "INSERT INTO VISTA_PATIENTS (MRN, DATE_ESTABLISHED, DATE_OF_DEATH, DOB, SEX, DATE_OF_LAST_UPDATE,NATIONALITY,PROVINCE,FILE_ACTIVE,FILE_TEMPORARY,MARITAL_STATUS) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"
                cursor.execute(insert_query, (my_optimus.encode(row.MRN), row.DATE_ESTABLISHED, row.DATE_OF_DEATH, row.DOB, row.SEX, row.DATE_OF_LAST_UPDATE,row.NATIONALITY,row.PROVINCE,row.FILE_ACTIVE,row.FILE_TEMPORARY,row.MARITAL_STATUS))
               
                insert_count+=1

        
            conn.commit()    
        cursor.close()
        print(f"Inserted rows: {insert_count}")
        print(f"Updated rows: {update_count}")
    
    
def main_function():
        try:
            extract_data()
            conn = establish_connection()
        except Exception as e:  
            print(f"Error encountered: {e}. Retrying in 10 minutes...")
            time.sleep(600)
            main_function()
#             if conn:
        try:
            data_list=parse_file('output_patient_list.txt', conn)
            insert_data_(conn, data_list, columns)  

        except Exception as e:  
            print(f"Error encountered: {e}. Retrying in 10 minutes...")
            time.sleep(600)
            main_function()
#         finally:
#             close_connection(conn)
        print("The End")
        print(f"Success! The next loop will start in 1 Hour...")
        time.sleep(3600)
        clear_output(wait=True)
if __name__ == "__main__":
    while True:
        try:
            main_function()
        except Exception as e:
            
            print(f"Error encountered: {e}. Retrying in 10 minutes...")
            time.sleep(600)
            main_function()
